# TP Chapitre - Entraînez-vous : implémentez une validation croisée

This the notebook where I constructed the method, step by step, so there are probably repeated parts of code.

In [1]:
data_path = "~/Documents/openclassroom/Fomation_ingenieur_ML/data/"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## sklearn module : 
from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import neighbors


In [2]:
df = pd.read_csv(data_path+'winequality-red+.csv', sep=";")

## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

#### comment je créée mes conteneurs pour extraire les données train/test ?
## PAR UN DICT GLOBAL ?
# original_data_split={"X_train":X_train,"X_test":X_test,
#                      "y_train":y_train, "y_test":y_test}
# original_data_split["X_train"]

# class DataSplit:
#    def __init__(self):
#     X_train = self.X_train
#     y_train = self.y_train
#     X_test = self.X_test
#     y_test = self.y_test
    
    
## EXTRACT X_train, X_test FROM X AND train_index :    
def extract_Xsplitted_data(X, train_index):    
#     train_index = X_train.index
    test_index = [(index not in train_index) for index in X.index]
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    return(X_train, X_test)

## STANDARDIZATION : 
def normalize_from_split(X, train_index):
    X_train, X_test = extract_Xsplitted_data(X, train_index)
    
    my_normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

    return(X_train_std, X_test_std)


###### LIST OF ARGUMENTS AT THE END 
clf_meth = neighbors.KNeighborsClassifier()

param_grid = {"n_neighbors" : np.arange(2,15,1)}

cv = 5

###### WHAT IT DOES : 
1. Create "cv"(=5 for ex) folds 
    
    1.1 Separate the observations in 5 parts that respect the global distrib (folds)

    1.2 Loop for fold_f in 1:5, 
            fit the clf_meth model on the Xcv_train = X_train[all fold - fold_f],
    
            loop for params in param_grid.keys() : 
            
                fit clf_meth(Xcv_train, ycv_train
            
            save results for prediction on Xcv_test = X_train[fold_f] 

2. define class methods : 
    
    2.1 cv_results_ : save mean results of models in the dict param_grid.key()
    
    2.2 best_params_ : the better parameter set in param_grid, in term of accuracy
    ... many other methods, see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In the chapter "Mettez en place un cadre de validation croisée", they explain all the approach of the Cross Validation, from the separation in kflod, with the "same" classification distribution (using sklearn.model_selection.KFold or sklearn.model_selection.StratifiedKFold). 

In a first time, I used those Methods to extract folds (##TODO : see if I have the time to implement it myself) and I computed the Cross Validation, to find the best hyper-parameters (in terms of acuracy) of my KNN.


## 1. Step by step construction :

This first part is decomposed step by step, then I agglomerate, and then I pass it into a sub class of scikit-learn "BaseCrossValidator" class

### 1.1 Initialization and preprocess

First, I initialize my parameters and define how to split and standardize my data : 

In [3]:
## READ DATA : 
df = pd.read_csv(data_path+'winequality-red+.csv', sep=";")

## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## SET ARGUMENTS/PARAMETERS :
clf_meth = neighbors.KNeighborsClassifier()
param_grid = {"n_neighbors" : np.arange(2,15,1)}
cv = 5

## INITIALIZATION OF DATA FOR CV: 
CV_X = X_train
CV_y = y_train 

As the standardization will be needed for each fold, let's implement it in a function :

In [4]:
## EXTRACT X_train, X_test FROM X AND train_index :    
def extract_Xsplitted_data(X, train_index):    
#     train_index = X_train.index
    test_index = [(index not in train_index) for index in X.index]
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    return(X_train, X_test)

## STANDARDIZATION : 
def normalize_from_split(X, train_index):
    X_train, X_test = extract_Xsplitted_data(X, train_index)
    
    my_normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

    return(X_train_std, X_test_std)


### 1.2 Define KNN method and loop iterators (folds and kwargs)
Secondly, I needed to separate into folds. To do so, I used the "naive" KFold method implemented in sklearn, and its improvement that take into account the repartition of y classes, "StratifiedKFold" : 

In [5]:
# kf = KFold(n_splits = 5)
# CV_split_iterator = kf.split(CV_X)

kf = model_selection.StratifiedKFold(n_splits = 5)
CV_split_iterator = kf.split(CV_X, y = CV_y)

Then, I call the functions defined above to extract the train and test samples and standardize with X_train data, and loop on the different folds splits : 

In [7]:

### LOOP ON FOLDS :
for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
    train_index = CV_X.index[CV_train_range_index]
    test_index = CV_X.index[CV_test_range_index] 
    
    ## NORMALIZE :
    CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
    ## GET y SPLIT : 
    CV_y_train, CV_y_test = y[train_index], y[test_index]
    

And within each iteration, I'll need to loop again, on the parameters of the KNN models, contained in "param_grid". To do so, the idea is to transform the dictionnary of lists into a list of dictionnary, mapping every possible values. The "model_selection.ParameterGrid()" exactly does that : 

In [ ]:
# MAP THE DICT OF LIST INTO LIST OF DICT :
param_grid = {"a":{0,1,2}, "b":{3,4}}
param_dirg = model_selection.ParameterGrid(param_grid)
for kwargs in param_dirg :
    print(kwargs)

And I used the "kwargs" syntax to call the dictionary of parameters in KNN model :

In [ ]:
kwargs = {"n_neighbors":3, 'metric': 'euclidean'}

knn_clf = neighbors.KNeighborsClassifier(**param)
knn_clf.fit(X_train_std,y_train)

## 2. Cross validation :

### 2.1 Code summary

Ok, so, the essential code to launch the CV is the following: 

In [15]:
## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## SET ARGUMENTS/PARAMETERS :
clf_meth = neighbors.KNeighborsClassifier()
# param_grid = {"n_neighbors" : np.arange(2,15,1)}
param_grid = {"n_neighbors" : np.arange(2,15,1),"metric": {'euclidean', 'manhattan'}}

cv = 5

###############################################################
##TODO : toutes les variables "locales" sont préfixées de 'CV_' 

## INITIALIZATION : 
CV_X = X_train
CV_y = y_train 

## SET FOLDS : 
kf = model_selection.StratifiedKFold(n_splits = 5)
CV_split_iterator = kf.split(CV_X, y = CV_y)
## MAP THE DICT OF LIST INTO LIST OF DICT :
param_dirg = model_selection.ParameterGrid(param_grid)

### LOOP ON FOLDS :
for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
    train_index = CV_X.index[CV_train_range_index]
    test_index = CV_X.index[CV_test_range_index] 
    
    ## NORMALIZE :
    CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
    ## GET y SPLIT : 
    CV_y_train, CV_y_test = y[train_index], y[test_index]
    
    ### LOOP ON PARAM NAMES (HERE ONLY 1)
    for kwargs in param_dirg :
        knn_clf = neighbors.KNeighborsClassifier(**kwargs)
        ## FIT KNN on X_train AND PREDICT ON X_test :
        knn_clf.fit(CV_X_train,CV_y_train)
        
        accuracy = knn_clf.score(CV_X_test, CV_y_test)
    

### 2.2 Save results in the procedure : 
But our aim is to extract some information from it. As an example, the sklearn method I aim to mimique as an attribute "CV_results_", with the keys : 

           'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 
           'param_metric', 'param_n_neighbors', 'params', 'split0_test_score',
           'split1_test_score', 'split2_test_score', 'split3_test_score',
           'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'
and as an example, with a 
it returns : 
        
     {'mean_fit_time': array([0.00328555, 0.00298281, 0.00299325, 0.0029798 ]),
     'std_fit_time': array([3.45699297e-04, 1.59457825e-04, 9.06425422e-05, 4.99484135e-05]),
     'mean_score_time': array([0.01436076, 0.01277046, 0.01385365, 0.01308317]),
     'std_score_time': array([0.00377103, 0.00024649, 0.00123275, 0.00020375]),
     'param_metric': masked_array(data=['euclidean', 'euclidean', 'manhattan', 'manhattan'],
                                  mask=[False, False, False, False],
                                  fill_value='?', dtype=object),
     'param_n_neighbors': masked_array(data=[10, 15, 10, 15],
                                       mask=[False, False, False, False],
                                       fill_value='?', dtype=object),
     'params':[{'metric': 'euclidean', 'n_neighbors': 10},
               {'metric': 'euclidean', 'n_neighbors': 15},
               {'metric': 'manhattan', 'n_neighbors': 10},
               {'metric': 'manhattan', 'n_neighbors': 15}],
     'split0_test_score': array([0.96875   , 0.96875   , 0.96428571, 0.96875   ]),
     'split1_test_score': array([0.96428571, 0.96428571, 0.96428571, 0.96428571]),
     'split2_test_score': array([0.96428571, 0.96428571, 0.95982143, 0.96428571]),
     'split3_test_score': array([0.96428571, 0.96428571, 0.96428571, 0.96428571]),
     'split4_test_score': array([0.96860987, 0.96860987, 0.96412556, 0.96860987]),
     'mean_test_score': array([0.9660434 , 0.9660434 , 0.96336083, 0.9660434 ]),
     'std_test_score': array([0.00215317, 0.00215317, 0.00177079, 0.00215317]),
     'rank_test_score': array([1, 1, 4, 1], dtype=int32)}
     
But I can first just extract the accuracy mean (that is what is expected !) 

In [55]:
## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
threshold = np.floor(df.quality.mean())
y = np.where(df.quality < threshold, 0,1) 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## SET ARGUMENTS/PARAMETERS :
clf_meth = neighbors.KNeighborsClassifier()
# NUMBER OF FOLDS :
cv = 5
param_grid = {"n_neighbors" : np.arange(2,15,1)}
# param_grid = {"n_neighbors" : np.arange(2,15,1),"metric": {'euclidean', 'manhattan'}}
# MAP THE DICT OF LIST INTO LIST OF DICT :
param_dirg = model_selection.ParameterGrid(param_grid)

###############################################################
##TODO : toutes les variables "locales" sont préfixées de 'CV_' 

## INITIALIZATION : 
CV_X = X_train
CV_y = y_train 

index = [("fold"+str(k+1)) for k in range(cv)]
k=0
columns = [("_".join(str(val) for val in kwargs.values())) for kwargs in param_dirg]
accuracy_df = pd.DataFrame(index = index, columns = columns )

## SET FOLDS : 
kf = model_selection.StratifiedKFold(n_splits = 5)
CV_split_iterator = kf.split(CV_X, y = CV_y)

### LOOP ON FOLDS :
for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
    k+=1
    index_name = "fold"+str(k)
    train_index = CV_X.index[CV_train_range_index]
    test_index = CV_X.index[CV_test_range_index] 
    
    ## NORMALIZE :
    CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
    ## GET y SPLIT : 
    CV_y_train, CV_y_test = y[train_index], y[test_index]
    
    ### LOOP ON PARAM NAMES (HERE ONLY 1)
    for kwargs in param_dirg :
        ## FIT KNN on X_train AND PREDICT ON X_test :
        knn_clf = neighbors.KNeighborsClassifier(**kwargs)
        knn_clf.fit(CV_X_train,CV_y_train)
        
        
        column_name = "_".join(str(val) for val in kwargs.values())
        accuracy_df.loc[index_name, column_name] = knn_clf.score(CV_X_test, CV_y_test)
    

In [66]:
accuracy_df.columns[0], accuracy_df.mean(axis = 0)[0], accuracy_df.std(axis = 0)[0]

('manhattan_2', 0.9276145099295323, 0.016215779492600276)

Now, the graal would have been to create a results class, that contains the different dataFrames such that accuracy_df, but also time ... that would be filled given a index_name (=fold) and column_name (=param grid product) ... See if I have time to do it. 

# 3. TP " Sélectionnez le nombre de voisins dans un kNN" with my CV : 

In [81]:
data = X_train, y_train 
## SET ARGUMENTS/PARAMETERS :
clf_meth = neighbors.KNeighborsClassifier()
# NUMBER OF FOLDS :
cv = 5
param_grid = {"n_neighbors" : np.arange(2,15,1)}
# param_grid = {"n_neighbors" : np.arange(2,15,1),"metric": {'euclidean', 'manhattan'}}

def my_CV(data, param_grid, clf_meth) :
    # MAP THE DICT OF LIST INTO LIST OF DICT :
    param_dirg = model_selection.ParameterGrid(param_grid)

    ###############################################################
    ##TODO : toutes les variables "locales" sont préfixées de 'CV_' 

    ## INITIALIZATION : 
    CV_X, CV_y = data 
#     CV_X = X_train
#     CV_y = y_train 

    index = [("fold"+str(k+1)) for k in range(cv)]
    k=0
    columns = [("_".join(str(val) for val in kwargs.values())) for kwargs in param_dirg]
    accuracy_df = pd.DataFrame(index = index, columns = columns )

    ## SET FOLDS : 
    kf = model_selection.StratifiedKFold(n_splits = 5)
    CV_split_iterator = kf.split(CV_X, y = CV_y)

    ### LOOP ON FOLDS :
    for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
        k+=1
        index_name = "fold"+str(k)
        train_index = CV_X.index[CV_train_range_index]
        test_index = CV_X.index[CV_test_range_index] 

        ## NORMALIZE :
        CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
        ## GET y SPLIT : 
        CV_y_train, CV_y_test = y[train_index], y[test_index]

        ### LOOP ON PARAM NAMES (HERE ONLY 1)
        for kwargs in param_dirg :
            ## FIT KNN on X_train AND PREDICT ON X_test :
            knn_clf = neighbors.KNeighborsClassifier(**kwargs)
            knn_clf.fit(CV_X_train,CV_y_train)


            column_name = "_".join(str(val) for val in kwargs.values())
            accuracy_df.loc[index_name, column_name] = knn_clf.score(CV_X_test, CV_y_test)
            
    return pd.DataFrame((accuracy_df.columns, accuracy_df.mean(axis = 0), accuracy_df.std(axis = 0)), index = ['n_neighbors', 'mean','std'])

res= my_CV(data, param_grid, clf_meth)

for col_index in res.columns:
    mean = res.loc["mean",col_index]
    std = res.loc["std",col_index]
    params = res.loc["n_neighbors",col_index]
    print("accuracy = %0.3f (+/-%0.3f) for n_neighbors = %s" %(mean, 2*std, params))


accuracy = 0.923 (+/-0.035) for n_neighbors = 2
accuracy = 0.959 (+/-0.015) for n_neighbors = 3
accuracy = 0.956 (+/-0.023) for n_neighbors = 4
accuracy = 0.962 (+/-0.010) for n_neighbors = 5
accuracy = 0.961 (+/-0.017) for n_neighbors = 6
accuracy = 0.963 (+/-0.015) for n_neighbors = 7
accuracy = 0.963 (+/-0.015) for n_neighbors = 8
accuracy = 0.965 (+/-0.007) for n_neighbors = 9
accuracy = 0.962 (+/-0.010) for n_neighbors = 10
accuracy = 0.966 (+/-0.005) for n_neighbors = 11
accuracy = 0.965 (+/-0.007) for n_neighbors = 12
accuracy = 0.965 (+/-0.007) for n_neighbors = 13
accuracy = 0.965 (+/-0.007) for n_neighbors = 14
